In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import accuracy_score

In [3]:
from scipy.io import arff
import pandas as pd

data, meta = arff.loadarff('/content/drive/MyDrive/COGS118A_Final_Project/Data/EEG_Eye_State.arff')

df = pd.DataFrame(data)

print(df.shape)
df.head()

(14980, 15)


,AF3,F7,F3,FC5,T7,P7,O1,O2,P8,T8,FC6,F4,F8,AF4,eyeDetection
0,4329.23,4009.23,4289.23,4148.21,4350.26,4586.15,4096.92,4641.03,4222.05,4238.46,4211.28,4280.51,4635.90,4393.85,b'0'
1,4324.62,4004.62,4293.85,4148.72,4342.05,4586.67,4097.44,4638.97,4210.77,4226.67,4207.69,4279.49,4632.82,4384.10,b'0'
2,4327.69,4006.67,4295.38,4156.41,4336.92,4583.59,4096.92,4630.26,4207.69,4222.05,4206.67,4282.05,4628.72,4389.23,b'0'
3,4328.72,4011.79,4296.41,4155.90,4343.59,4582.56,4097.44,4630.77,4217.44,4235.38,4210.77,4287.69,4632.31,4396.41,b'0'
4,4326.15,4011.79,4292.31,4151.28,4347.69,4586.67,4095.90,4627.69,4210.77,4244.10,4212.82,4288.21,4632.82,4398.46,b'0'


In [4]:
label_col = df.columns[-1]   # last column is the label
print("Label column:", label_col)
print(df[label_col].head())
print(type(df[label_col].iloc[0]))

Label column: eyeDetection
0    b'0'
1    b'0'
2    b'0'
3    b'0'
4    b'0'
Name: eyeDetection, dtype: object
<class 'bytes'>


In [5]:
# Convert byte labels like b'0', b'1' into integer 0, 1
df[label_col] = df[label_col].apply(lambda v: int(str(v).replace("b'", "").replace("'", "")))
print(df[label_col].unique())
print(df.dtypes)

[0 1]
AF3             float64
F7              float64
F3              float64
FC5             float64
T7              float64
P7              float64
O1              float64
O2              float64
P8              float64
T8              float64
FC6             float64
F4              float64
F8              float64
AF4             float64
eyeDetection      int64
dtype: object


In [6]:
# X = all columns except label
X = df.drop(columns=[label_col])

# y = just the label column as a 1D array
y = df[label_col].values   # or df[label_col].to_numpy()

In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    train_size=0.8,
    stratify=y,       # keep class balance
    random_state=0
)

In [8]:
log_reg_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', LogisticRegression(max_iter=1000))
])

log_reg_pipeline.fit(X_train, y_train)  # this should now work

y_train_pred = log_reg_pipeline.predict(X_train)
y_test_pred = log_reg_pipeline.predict(X_test)

train_acc = accuracy_score(y_train, y_train_pred)
test_acc = accuracy_score(y_test, y_test_pred)

print("Train accuracy:", train_acc)
print("Test accuracy:", test_acc)

Train accuracy: 0.5856141522029372
Test accuracy: 0.5834445927903872


In [9]:
# Check class distribution
unique, counts = np.unique(y, return_counts=True)
for cls, cnt in zip(unique, counts):
    print(f"Class {cls}: {cnt} examples")

# Majority-class baseline accuracy
majority_class = unique[np.argmax(counts)]
baseline_acc = np.max(counts) / len(y)
print("Majority class:", majority_class)
print("Baseline (always predict majority) accuracy:", baseline_acc)

Class 0: 8257 examples
Class 1: 6723 examples
Majority class: 0
Baseline (always predict majority) accuracy: 0.5512016021361815


In [10]:
from sklearn.svm import SVC

svm_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', SVC(kernel='rbf'))
])

svm_pipeline.fit(X_train, y_train)

y_train_pred_svm = svm_pipeline.predict(X_train)
y_test_pred_svm = svm_pipeline.predict(X_test)

print("SVM Train accuracy:", accuracy_score(y_train, y_train_pred_svm))
print("SVM Test accuracy:", accuracy_score(y_test, y_test_pred_svm))

SVM Train accuracy: 0.6303404539385847
SVM Test accuracy: 0.6224966622162884


In [11]:
from sklearn.ensemble import RandomForestClassifier

rf_pipeline = Pipeline([
    ('clf', RandomForestClassifier(random_state=0))
])

rf_pipeline.fit(X_train, y_train)

y_train_pred_rf = rf_pipeline.predict(X_train)
y_test_pred_rf = rf_pipeline.predict(X_test)

print("RF Train accuracy:", accuracy_score(y_train, y_train_pred_rf))
print("RF Test accuracy:", accuracy_score(y_test, y_test_pred_rf))

RF Train accuracy: 1.0
RF Test accuracy: 0.9289052069425902


In [12]:
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

mlp_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', MLPClassifier(
        hidden_layer_sizes=(50,),
        max_iter=1000,         # allow more steps
        early_stopping=True,   # stop when validation score stops improving
        n_iter_no_change=10,   # patience
        random_state=0
    ))
])

mlp_pipeline.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('clf',
                 MLPClassifier(early_stopping=True, hidden_layer_sizes=(50,),
                               max_iter=1000, random_state=0))])

In [13]:
from sklearn.metrics import accuracy_score

y_train_pred_mlp = mlp_pipeline.predict(X_train)
y_test_pred_mlp = mlp_pipeline.predict(X_test)

print("MLP Train accuracy:", accuracy_score(y_train, y_train_pred_mlp))
print("MLP Test accuracy:", accuracy_score(y_test, y_test_pred_mlp))

MLP Train accuracy: 0.8022363150867824
MLP Test accuracy: 0.790720961281709


In [14]:
results_80_20 = {
    "LogisticRegression": (train_acc, test_acc),
    "SVM_RBF": (accuracy_score(y_train, y_train_pred_svm),
                accuracy_score(y_test, y_test_pred_svm)),
    "RandomForest": (accuracy_score(y_train, y_train_pred_rf),
                     accuracy_score(y_test, y_test_pred_rf)),
    "MLP": (accuracy_score(y_train, y_train_pred_mlp),
            accuracy_score(y_test, y_test_pred_mlp))
}

for name, (tr, te) in results_80_20.items():
    print(f"{name:16s} | Train: {tr:.3f} | Test: {te:.3f}")

LogisticRegression | Train: 0.586 | Test: 0.583
SVM_RBF          | Train: 0.630 | Test: 0.622
RandomForest     | Train: 1.000 | Test: 0.929
MLP              | Train: 0.802 | Test: 0.791


In [15]:
# Dictionary of models and their hyperparameter grids
models_and_parameters = {
    "LogisticRegression": (
        Pipeline([
            ('scaler', StandardScaler()),
            ('clf', LogisticRegression(max_iter=1000))
        ]),
        {
            'clf__C': [0.01, 0.1, 1, 10, 100]
        }
    ),
    "SVM_RBF": (
        Pipeline([
            ('scaler', StandardScaler()),
            ('clf', SVC(kernel='rbf', random_state=0))
        ]),
        {
            'clf__C': [0.1, 1, 10, 100],
            'clf__gamma': ['scale', 0.01, 0.1, 1]
        }
    ),
    "RandomForest": (
        Pipeline([
            ('clf', RandomForestClassifier(random_state=0))
        ]),
        {
            'clf__n_estimators': [100, 300],
            'clf__max_depth': [None, 10, 20],
            'clf__min_samples_leaf': [1, 5]
        }
    ),
    "MLP": (
        Pipeline([
            ('scaler', StandardScaler()),
            ('clf', MLPClassifier(
                max_iter=1000,
                early_stopping=True,
                n_iter_no_change=10,
                random_state=0
            ))
        ]),
        {
            'clf__hidden_layer_sizes': [(50,), (100,), (50, 50)],
            'clf__alpha': [0.0001, 0.001],
            'clf__learning_rate_init': [0.001, 0.01]
        }
    )
}

In [16]:
results = []

for name, (pipe, param_grid) in models_and_parameters.items():
    print(f"\n=== {name} ===")

    grid = GridSearchCV(
        estimator=pipe,
        param_grid=param_grid,
        scoring='accuracy',
        cv=5,          # 5-fold cross-validation
        n_jobs=-1,     # use all cores (you can change to 1 if issues)
        verbose=1
    )

    # Fit on training data only
    grid.fit(X_train, y_train)

    best_model = grid.best_estimator_
    best_params = grid.best_params_
    best_cv_score = grid.best_score_   # mean CV accuracy

    # Evaluate on train and test sets
    y_train_pred = best_model.predict(X_train)
    y_test_pred = best_model.predict(X_test)

    train_acc = accuracy_score(y_train, y_train_pred)
    test_acc = accuracy_score(y_test, y_test_pred)

    print("Best params:", best_params)
    print(f"Best CV accuracy: {best_cv_score:.3f}")
    print(f"Train accuracy:   {train_acc:.3f}")
    print(f"Test accuracy:    {test_acc:.3f}")

    results.append({
        "model": name,
        "best_params": best_params,
        "cv_accuracy": best_cv_score,
        "train_accuracy": train_acc,
        "test_accuracy": test_acc
    })

results_df = pd.DataFrame(results)
results_df


=== LogisticRegression ===
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Best params: {'clf__C': 100}
Best CV accuracy: 0.619
Train accuracy:   0.615
Test accuracy:    0.619

=== SVM_RBF ===
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Best params: {'clf__C': 100, 'clf__gamma': 1}
Best CV accuracy: 0.888
Train accuracy:   0.901
Test accuracy:    0.870

=== RandomForest ===
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Best params: {'clf__max_depth': None, 'clf__min_samples_leaf': 1, 'clf__n_estimators': 300}
Best CV accuracy: 0.925
Train accuracy:   1.000
Test accuracy:    0.930

=== MLP ===
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Best params: {'clf__alpha': 0.001, 'clf__hidden_layer_sizes': (50, 50), 'clf__learning_rate_init': 0.01}
Best CV accuracy: 0.874
Train accuracy:   0.919
Test accuracy:    0.883


,model,best_params,cv_accuracy,train_accuracy,test_accuracy
0,LogisticRegression,{'clf__C': 100},0.619411,0.614820,0.619159
1,SVM_RBF,"{'clf__C': 100, 'clf__gamma': 1}",0.887684,0.900617,0.869826
2,RandomForest,"{'clf__max_depth': None, 'clf__min_samples_lea...",0.925150,1.000000,0.930240
3,MLP,"{'clf__alpha': 0.001, 'clf__hidden_layer_sizes...",0.873916,0.918558,0.883178


In [17]:
def run_all_models_with_gridsearch(X_train, y_train, X_test, y_test,
                                   dataset_name="EEG_Eye_State",
                                   train_size=0.8,
                                   trial=0):
    models_and_parameters = {
        "LogisticRegression": (
            Pipeline([
                ('scaler', StandardScaler()),
                ('clf', LogisticRegression(max_iter=1000))
            ]),
            {
                'clf__C': [0.01, 0.1, 1, 10, 100]
            }
        ),
        "SVM_RBF": (
            Pipeline([
                ('scaler', StandardScaler()),
                ('clf', SVC(kernel='rbf', random_state=0))
            ]),
            {
                'clf__C': [0.1, 1, 10, 100],
                'clf__gamma': ['scale', 0.01, 0.1, 1]
            }
        ),
        "RandomForest": (
            Pipeline([
                ('clf', RandomForestClassifier(random_state=0))
            ]),
            {
                'clf__n_estimators': [100, 300],
                'clf__max_depth': [None, 10, 20],
                'clf__min_samples_leaf': [1, 5]
            }
        ),
        "MLP": (
            Pipeline([
                ('scaler', StandardScaler()),
                ('clf', MLPClassifier(
                    max_iter=1000,
                    early_stopping=True,
                    n_iter_no_change=10,
                    random_state=0
                ))
            ]),
            {
                'clf__hidden_layer_sizes': [(50,), (100,), (50, 50)],
                'clf__alpha': [0.0001, 0.001],
                'clf__learning_rate_init': [0.001, 0.01]
            }
        )
    }

    results = []

    for name, (pipe, param_grid) in models_and_parameters.items():
        print(f"\n=== {name} (train_size={train_size}, trial={trial}) ===")

        grid = GridSearchCV(
            estimator=pipe,
            param_grid=param_grid,
            scoring='accuracy',
            cv=5,
            n_jobs=-1,
            verbose=0
        )

        grid.fit(X_train, y_train)

        best_model = grid.best_estimator_
        best_params = grid.best_params_
        best_cv_score = grid.best_score_

        y_train_pred = best_model.predict(X_train)
        y_test_pred = best_model.predict(X_test)

        train_acc = accuracy_score(y_train, y_train_pred)
        test_acc = accuracy_score(y_test, y_test_pred)

        print("  Best params:", best_params)
        print(f"  CV accuracy:   {best_cv_score:.3f}")
        print(f"  Train accuracy:{train_acc:.3f}")
        print(f"  Test accuracy: {test_acc:.3f}")

        results.append({
            "dataset": dataset_name,
            "train_size": train_size,
            "trial": trial,
            "model": name,
            "best_params": best_params,
            "cv_accuracy": best_cv_score,
            "train_accuracy": train_acc,
            "test_accuracy": test_acc
        })

    return pd.DataFrame(results)

In [ ]:
train_sizes = [0.2, 0.5, 0.8]
trials = [0, 1, 2]  # 3 trials
all_results = []

for ts in train_sizes:
    for t in trials:
        print(f"\n\n### Running experiments: train_size={ts}, trial={t} ###")

        X_train, X_test, y_train, y_test = train_test_split(
            X, y,
            train_size=ts,
            stratify=y,
            random_state=t
        )

        df_results = run_all_models_with_gridsearch(
            X_train, y_train, X_test, y_test,
            dataset_name="EEG_Eye_State",
            train_size=ts,
            trial=t
        )

        all_results.append(df_results)

# Concatenate everything into one big table
eeg_eye_results = pd.concat(all_results, ignore_index=True)
eeg_eye_results



### Running experiments: train_size=0.2, trial=0 ###

=== LogisticRegression (train_size=0.2, trial=0) ===
  Best params: {'clf__C': 10}
  CV accuracy:   0.640
  Train accuracy:0.644
  Test accuracy: 0.640

=== SVM_RBF (train_size=0.2, trial=0) ===
  Best params: {'clf__C': 10, 'clf__gamma': 1}
  CV accuracy:   0.932
  Train accuracy:1.000
  Test accuracy: 0.937

=== RandomForest (train_size=0.2, trial=0) ===
  Best params: {'clf__max_depth': None, 'clf__min_samples_leaf': 1, 'clf__n_estimators': 300}
  CV accuracy:   0.872
  Train accuracy:1.000
  Test accuracy: 0.873

=== MLP (train_size=0.2, trial=0) ===
  Best params: {'clf__alpha': 0.0001, 'clf__hidden_layer_sizes': (50, 50), 'clf__learning_rate_init': 0.01}
  CV accuracy:   0.909
  Train accuracy:0.982
  Test accuracy: 0.918


### Running experiments: train_size=0.2, trial=1 ###

=== LogisticRegression (train_size=0.2, trial=1) ===
  Best params: {'clf__C': 10}
  CV accuracy:   0.658
  Train accuracy:0.662
  Test accuracy: 0.63

In [ ]:
eeg_eye_summary = (
    eeg_eye_results
    .groupby(["model", "train_size"])
    .agg(
        mean_test_acc=("test_accuracy", "mean"),
        std_test_acc=("test_accuracy", "std"),
        mean_train_acc=("train_accuracy", "mean"),
        mean_cv_acc=("cv_accuracy", "mean")
    )
    .reset_index()
)

eeg_eye_summary

In [ ]:
results_dir = "/content/drive/MyDrive/COGS118A_Final_Project/Results"

import os
os.makedirs(results_dir, exist_ok=True)

eeg_eye_results.to_csv(f"{results_dir}/eeg_eye_results_detailed.csv", index=False)
eeg_eye_summary.to_csv(f"{results_dir}/eeg_eye_results_summary.csv", index=False)

print("Saved:")
print(f" - {results_dir}/eeg_eye_results_detailed.csv")
print(f" - {results_dir}/eeg_eye_results_summary.csv")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid", font_scale=1.2)

In [ ]:
plt.figure(figsize=(10,6))
sns.lineplot(
    data=eeg_eye_summary,
    x="train_size",
    y="mean_test_acc",
    hue="model",
    marker="o"
)
plt.title("EEG Eye State — Test Accuracy vs Train Size")
plt.xlabel("Train Size")
plt.ylabel("Mean Test Accuracy")
plt.ylim(0.5, 1.0)
plt.legend(title="Model")
plt.savefig("/content/drive/MyDrive/COGS118A_Final_Project/Plots/eye_state_testacc_vs_trainsize.png", dpi=300, bbox_inches='tight')
plt.show()